In [3]:
#untuk mengakses google drive
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
#import fitur
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [5]:
#mengakses data movielens dari google drive
import pandas as pd
movies = pd.read_csv('/gdrive/My Drive/Movielens/movies.csv',encoding='Latin1')
Ratings = pd.read_csv('/gdrive/My Drive/Movielens/ratings.csv')
Tags = pd.read_csv('/gdrive/My Drive/Movielens/tags.csv',encoding='Latin1')

Mean = Ratings.groupby(by="userId",as_index=False)['rating'].mean()
Rating_avg = pd.merge(Ratings,Mean,on='userId')
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']
Rating_avg.head()

,userId,movieId,rating_x,timestamp,rating_y,adg_rating
0,1,2,3.5,1112486027,3.742857,-0.242857
1,1,29,3.5,1112484676,3.742857,-0.242857
2,1,32,3.5,1112484819,3.742857,-0.242857
3,1,47,3.5,1112484727,3.742857,-0.242857
4,1,50,3.5,1112484580,3.742857,-0.242857


In [6]:
#menampilkan data movie
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
#menampilkan data tags
Tags.head()

,userId,movieId,tag,timestamp
0,18,4141,Mark Waters,1240597180
1,65,208,dark hero,1368150078
2,65,353,dark hero,1368150079
3,65,521,noir thriller,1368149983
4,65,592,dark hero,1368150078


In [8]:
#menampilkan pivot table berdasarkan rating x (NaN = Not aa Number)
check = pd.pivot_table(Rating_avg,values='rating_x',index='userId',columns='movieId')
check.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,127096,127098,127110,127166,127196,127212,127222,127224,127302,127319,127323,127881,128356,128358,128488,128510,128520,128594,128601,128622,128648,128686,128715,128736,128832,128842,129030,129233,129235,129303,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.5,NaN,NaN,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
#menampilkan pivot table berdasarkan adg rating (NaN = Not aa Number)
final = pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='movieId')
final.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,127096,127098,127110,127166,127196,127212,127222,127224,127302,127319,127323,127881,128356,128358,128488,128510,128520,128594,128601,128622,128648,128686,128715,128736,128832,128842,129030,129233,129235,129303,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,-0.242857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.242857,NaN,NaN,-0.242857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.122995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.122995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.122995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,-0.571429,NaN,NaN,NaN,0.428571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.571429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,-1.272727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.727273,NaN,NaN,NaN,NaN,NaN,-1.272727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# mengganti NaN berdasarkan  average Movie 
final_movie = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [11]:
#menampilkan data pivot tabel movie yang NaN sudah menjadi average movie 
final_movie.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,127096,127098,127110,127166,127196,127212,127222,127224,127302,127319,127323,127881,128356,128358,128488,128510,128520,128594,128601,128622,128648,128686,128715,128736,128832,128842,129030,129233,129235,129303,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.354987,-0.242857,-0.356159,-0.504011,-0.374364,0.288623,-0.145776,-0.097414,-0.440951,-0.118551,0.141705,-0.751099,-0.182008,-0.099404,-0.71644,0.305447,0.340405,-0.100596,-0.900737,-0.580073,0.079819,-0.129315,-0.345156,-0.306375,0.145216,0.132519,-0.15529,0.393242,-0.242857,0.086523,-0.181294,-0.242857,-0.56053,0.075733,0.16026,0.328567,-0.27574,-0.474628,-0.128747,0.210787,...,-1.10241,-0.160781,-1.486486,-1.986486,-1.986486,-1.986486,0.418429,-0.581571,-0.442161,0.632948,-0.867052,-0.42403,-1.564737,-0.064737,-0.972057,0.333667,0.398305,-2.313291,-0.166333,0.333667,-0.875697,-0.749428,-0.052863,-2.459276,0.25,-0.397436,-1.486486,-0.875697,0.315789,-0.10119,0.040724,-0.162266,-0.250513,-1.986486,-1.597955,-0.375697,0.557331,0.124595,-1.135011,0.402045
2,0.354987,-0.269639,0.000000,-0.504011,-0.374364,0.288623,-0.145776,-0.097414,-0.440951,-0.118551,0.141705,-0.751099,-0.182008,-0.099404,-0.71644,0.305447,0.340405,-0.100596,-0.900737,-0.580073,0.079819,-0.129315,-0.345156,-0.306375,0.145216,0.132519,-0.15529,0.393242,0.456845,0.086523,-0.181294,0.322479,-0.56053,0.075733,0.16026,0.328567,-0.27574,-0.474628,-0.128747,0.210787,...,-1.10241,-0.160781,-1.486486,-1.986486,-1.986486,-1.986486,0.418429,-0.581571,-0.442161,0.632948,-0.867052,-0.42403,-1.564737,-0.064737,-0.972057,0.333667,0.398305,-2.313291,-0.166333,0.333667,-0.875697,-0.749428,-0.052863,-2.459276,0.25,-0.397436,-1.486486,-0.875697,0.315789,-0.10119,0.040724,-0.162266,-0.250513,-1.986486,-1.597955,-0.375697,0.557331,0.124595,-1.135011,0.402045
3,-0.122995,-0.269639,-0.356159,-0.504011,-0.374364,0.288623,-0.145776,-0.097414,-0.440951,-0.118551,0.141705,-0.751099,-0.182008,-0.099404,-0.71644,0.305447,0.340405,-0.100596,-0.900737,-0.580073,0.079819,-0.129315,-0.345156,-1.122995,0.145216,0.132519,-0.15529,0.393242,0.456845,0.086523,-0.181294,-0.122995,-0.56053,0.075733,0.16026,0.328567,-0.27574,-0.474628,-0.128747,0.210787,...,-1.10241,-0.160781,-1.486486,-1.986486,-1.986486,-1.986486,0.418429,-0.581571,-0.442161,0.632948,-0.867052,-0.42403,-1.564737,-0.064737,-0.972057,0.333667,0.398305,-2.313291,-0.166333,0.333667,-0.875697,-0.749428,-0.052863,-2.459276,0.25,-0.397436,-1.486486,-0.875697,0.315789,-0.10119,0.040724,-0.162266,-0.250513,-1.986486,-1.597955,-0.375697,0.557331,0.124595,-1.135011,0.402045
4,0.354987,-0.269639,-0.356159,-0.504011,-0.374364,-0.571429,-0.145776,-0.097414,-0.440951,0.428571,0.141705,-0.751099,-0.182008,-0.099404,-0.71644,0.305447,0.340405,-0.100596,-0.571429,-0.580073,0.079819,-0.129315,-0.345156,-0.306375,0.145216,0.132519,-0.15529,0.393242,0.456845,0.086523,-0.181294,-2.571429,-0.56053,0.075733,0.16026,0.328567,-0.27574,-0.474628,-0.128747,0.210787,...,-1.10241,-0.160781,-1.486486,-1.986486,-1.986486,-1.986486,0.418429,-0.581571,-0.442161,0.632948,-0.867052,-0.42403,-1.564737,-0.064737,-0.972057,0.333667,0.398305,-2.313291,-0.166333,0.333667,-0.875697,-0.749428,-0.052863,-2.459276,0.25,-0.397436,-1.486486,-0.875697,0.315789,-0.10119,0.040724,-0.162266,-0.250513,-1.986486,-1.597955,-0.375697,0.557331,0.124595,-1.135011,0.402045
5,0.354987,-1.272727,-0.356159,-0.504011,-0.374364,0.288623,-0.145776,-0.097414,-0.440951,-0.118551,0.727273,-0.751099,-0.182008,-0.099404,-0.71644,0.305447,-1.272727,-0.100596,-0.900737,-0.580073,0.079819,-0.129315,-0.345156,-0.306375,0.145216,0.132519,-0.15529,0.393242,0.456845,0.086523,-0.181294,0.322479,-0.56053,0.075733,0.16026,0.328567,-0.27574,-0.474628,-0.128747,0.210787,...,-1.10241,-0.160781,-1.486486,-1.986486,-1.986486,-1

In [12]:
#menampilkan data user tabel movie yang NaN sudah menjadi average movie 
final_user.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,127096,127098,127110,127166,127196,127212,127222,127224,127302,127319,127323,127881,128356,128358,128488,128510,128520,128594,128601,128622,128648,128686,128715,128736,128832,128842,129030,129233,129235,129303,129350,129354,129428,129707,130052,130073,130219,130462,130490,130642
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,-2.537653e-17,-2.428571e-01,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.428571e-01,-2.537653e-17,-2.537653e-17,-2.428571e-01,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,...,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17,-2.537653e-17
2,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3,-1.229947e-01,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-1.122995e+00,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-1.229947e-01,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,...,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17,-4.749617e-17
4,-2.220446e-16,-2.220446e-16,-2.220446e-16,-2.220446e-16,-2.220446e-16,-5.714286e-01,-2.220446e-16,-2.220446e-16,-2.220446e-16,4.285714e-01,-2.220446e-16,-2.220446e-16,-2.220446e-16,-2.220446e-16,-2.220446e-16,

In [13]:
# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,7081,7082,7083,7084,7085,7086,7087,7088,7089,7090,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.025079,0.052611,0.031839,0.057268,-0.004678,0.031860,-0.007859,8.064088e-04,0.071578,0.057648,-0.011886,0.010066,-0.036223,-0.000247,-0.009086,-0.013407,-0.037949,0.005909,-2.828999e-02,-0.005297,0.013937,0.001311,0.034855,-0.018447,-0.013679,-0.011149,0.022086,0.026893,0.004838,0.070009,0.005440,-0.007201,-0.014734,0.024170,-1.490804e-03,4.442945e-03,-0.010239,0.075630,-0.011302,...,0.077354,-0.027122,0.072369,0.014344,-0.021108,0.017847,0.001760,0.024122,0.011307,-4.984855e-02,-1.178681e-02,-0.044869,-0.031201,-0.008473,6.842257e-04,0.015794,0.045256,-0.002616,1.668808e-30,-0.006953,6.393930e-03,0.094175,1.397370e-01,0.064987,0.001721,-0.030752,-0.013907,0.006160,0.063903,0.096938,0.013262,1.755313e-02,-0.008575,6.611135e-02,-0.000905,0.101994,0.044777,-4.135166e-02,-0.013052,0.018360
2,0.025079,0.000000,0.064919,0.025317,0.052187,0.035336,0.088485,0.039048,3.081488e-33,0.013856,0.022194,0.014029,0.012436,0.034760,-0.005281,0.008149,-0.036275,-0.062727,0.025380,6.162976e-33,0.112838,0.035930,-0.009658,0.028794,-0.013455,0.046555,0.015836,-0.046495,0.039167,-0.007531,0.012960,0.013784,-0.026066,0.034240,-0.032750,-3.081488e-33,3.081488e-33,0.018276,0.041835,0.003834,...,0.077005,0.016352,0.051625,0.004630,0.018076,0.018203,0.028305,0.012519,0.004852,-8.668315e-02,6.162976e-33,-0.062989,-0.024304,-0.030393,9.244464e-33,0.006501,0.029848,0.000000,-3.081488e-33,0.008429,-1.540744e-33,0.028588,-3.081488e-33,0.031337,0.026783,0.008710,-0.004775,0.063631,-0.045374,0.005873,0.011279,3.081488e-33,0.046820,1.170373e-02,0.001400,0.057578,0.006336,1.377168e-01,0.034293,0.006262
3,0.052611,0.064919,0.000000,0.015300,-0.008025,-0.024011,0.012590,0.012737,2.177168e-02,0.026654,-0.007203,0.009719,0.016695,-0.036075,0.004494,-0.006407,-0.002052,-0.024818,0.013101,8.005923e-02,0.097310,0.087782,0.036831,0.099899,0.023738,0.085239,0.007171,0.015019,0.018737,0.026561,0.009856,0.050238,0.014687,0.075296,-0.001972,-5.288265e-03,-2.189221e-02,-0.008883,0.028310,0.044294,...,0.121645,0.033334,0.074077,0.090765,0.034957,0.060586,0.017344,0.070209,0.049325,-6.206319e-02,-4.887558e-03,-0.010850,0.022831,0.006649,5.117765e-02,0.023179,0.060912,0.023198,1.476707e-03,-0.008606,6.054415e-02,-0.015673,3.738557e-02,0.081272,0.004387,0.077711,-0.002044,0.127830,0.016690,0.010706,0.017999,1.751704e-02,-0.015580,5.513281e-02,-0.012029,0.008905,0.036820,1.587831e-02,0.045976,0.011999
4,0.031839,0.025317,0.015300,0.000000,0.044112,-0.050693,0.021210,0.158575,1.485647e-02,-0.025345,-0.010961,0.151617,-0.045477,-0.024007,0.088950,0.014951,-0.058731,-0.014950,0.188833,-4.842615e-03,-0.024109,0.008242,-0.081577,-0.017955,0.030717,0.031746,0.018588,0.037798,0.209188,0.048707,-0.003730,-0.010940,-0.044873,0.038277,0.006278,-3.869073e-29,-1.477951e-29,0.012788,0.096028,-0.023379,...,0.035351,0.018767,0.039591,-0.003720,0.036454,0.001934,-0.050823,-0.006225,0.061259,-7.771926e-30,1.638063e-02,0.007785,-0.001525,-0.038370,2.401045e-02,0.003730,0.024086,-0.003570,-2.101416e-02,-0.001303,2.008916e-29,-0.006457,2.963522e-29,-0.006173,-0.034421,0.002499,0.054558,0.048269,0.026038,-0.027448,-0.016181,1.071634e-02,0.201465,-1.858145e-29,0.014060,0.018712,-0.003757,2.442954e-29,-0.127582,0.032342
5,0.057268,0.052187,-0.008025,0.044112,0.000000,0.050837,0.080559,-0.004609,8.069115e-30,0.015948,0.028558,-0.052566,0.018453,0.011929,-0.025023,-0.012198,0.055418,-0.007946,-0.036346,3.520856e-02,0.019081,-0.011730,0.007411,0.036002,0.052562,0.043376,0.025890,-0.032065,0.012487,-0.005778,0.010001,0.074037,0.001739,0.087971,-0.001583,1.350912e-29,5.277805e-30,0.049359,0.

In [14]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index
similarity_with_movie.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,7081,7082,7083,7084,7085,7086,7087,7088,7089,7090,7091,7092,7093,7094,7095,7096,7097,7098,7099,7100,7101,7102,7103,7104,7105,7106,7107,7108,7109,7110,7111,7112,7113,7114,7115,7116,7117,7118,7119,7120
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0.000000,0.993645,0.989068,0.995953,0.993566,0.995469,0.985177,0.993894,0.994673,0.996406,0.957929,0.996267,0.993050,0.986913,0.996201,0.992584,0.996333,0.990590,0.995112,0.996183,0.988833,0.991328,0.989723,0.963328,0.993295,0.995149,0.993930,0.994326,0.991238,0.994862,0.957632,0.995293,0.991430,0.993746,0.991144,0.996177,0.996228,0.990339,0.996509,0.991215,...,0.994513,0.996518,0.982679,0.993984,0.994869,0.995651,0.994959,0.983744,0.995568,0.995147,0.995717,0.992557,0.995380,0.995056,0.995694,0.994002,0.988730,0.996672,0.994843,0.991577,0.996490,0.985746,0.996628,0.993903,0.989814,0.991484,0.979983,0.995173,0.996237,0.991246,0.992366,0.996391,0.990803,0.996072,0.994978,0.989353,0.987931,0.995011,0.995373,0.996025
2,0.993645,0.000000,0.988834,0.995416,0.993240,0.995189,0.985937,0.994043,0.994434,0.995516,0.957547,0.995798,0.992858,0.986968,0.995473,0.992441,0.995802,0.989755,0.994721,0.995937,0.989375,0.991337,0.989409,0.963335,0.992468,0.995263,0.993731,0.993607,0.991167,0.994368,0.956686,0.995075,0.990972,0.993545,0.990297,0.995848,0.995818,0.989731,0.996071,0.990900,...,0.994139,0.996049,0.982715,0.993943,0.994803,0.995299,0.994685,0.983732,0.994698,0.994470,0.995283,0.991874,0.994816,0.994377,0.995407,0.993703,0.987214,0.996261,0.994463,0.991248,0.996018,0.985458,0.996021,0.993667,0.989719,0.991486,0.979727,0.995379,0.995655,0.990087,0.991986,0.996015,0.990378,0.995575,0.994238,0.989081,0.986762,0.995841,0.995259,0.995537
3,0.989068,0.988834,0.000000,0.990749,0.987702,0.990023,0.980075,0.988536,0.989736,0.990406,0.952553,0.991088,0.987691,0.980892,0.990624,0.987078,0.990892,0.984807,0.989907,0.991383,0.984147,0.986162,0.985078,0.960495,0.987806,0.990091,0.988728,0.989011,0.985717,0.989589,0.952096,0.989905,0.986222,0.988812,0.985484,0.990914,0.990795,0.984408,0.991162,0.986073,...,0.989928,0.991203,0.978089,0.989715,0.989765,0.990412,0.989869,0.978455,0.990434,0.989450,0.990297,0.987109,0.989933,0.989575,0.990615,0.988665,0.982588,0.991210,0.989588,0.986246,0.991160,0.979284,0.991190,0.988758,0.984274,0.987079,0.973675,0.990674,0.990963,0.985112,0.987065,0.991159,0.984923,0.990694,0.988997,0.982924,0.982128,0.989760,0.990340,0.990764
4,0.995953,0.995416,0.990749,0.000000,0.995190,0.996885,0.986879,0.995924,0.996360,0.997529,0.958593,0.998230,0.994503,0.988287,0.997562,0.994123,0.997791,0.992070,0.997265,0.997749,0.990702,0.992855,0.991153,0.964933,0.994617,0.996854,0.995502,0.995911,0.993790,0.996509,0.958556,0.997028,0.992869,0.995526,0.992488,0.997771,0.997755,0.991566,0.998270,0.992645,...,0.996411,0.997909,0.984675,0.995819,0.996612,0.997188,0.996459,0.985490,0.997298,0.996642,0.997211,0.994366,0.997041,0.996490,0.997333,0.995473,0.989470,0.998311,0.996438,0.993128,0.997948,0.986942,0.997950,0.995586,0.991361,0.993379,0.982392,0.997175,0.997649,0.992207,0.993738,0.997971,0.993146,0.997498,0.996241,0.990380,0.989021,0.996667,0.996561,0.997543
5,0.993566,0.993240,0.987702,0.995190,0.000000,0.995004,0.985308,0.993087,0.994096,0.995203,0.957284,0.995299,0.992181,0.986200,0.995287,0.991709,0.995747,0.990154,0.994136,0.995592,0.988343,0.990279,0.989391,0.963479,0.992396,0.994736,0.993342,0.993533,0.990220,0.994057,0.956263,0.995323,0.990916,0.993536,0.990383,0.995503,0.995487,0.989931,0.995841,0.990764,...,0.992905,0.995839,0.981863,0.993458,0.994311,0.994874,0.994355,0.982958,0.994698,0.994304,0.995017,0.992011,0.994585,0.994492,0.995194,0.993700,0.987466,0.995953,0.994225,0.990953,0.995676,0.985185,0.995682,0.993279,0.989245,0.990651,0.979199,0.994712,0.995598,0.990486,0.991692,0.995676,0.990518,0.995262,0.995181,0.987

In [0]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [16]:
# menampilkan top 30 algoritma KNN for each user
sim_user_30_u = find_n_neighbours(similarity_with_user,30)
sim_user_30_u.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,3083,1385,1596,179,211,1273,194,83,6310,5554,5845,5874,6421,4499,4746,4099,4461,1903,3362,526,6541,734,2452,1962,4079,5854,5616,1687,2207,4336
2,5295,6062,213,5974,1004,3056,4612,1941,6944,5126,554,1794,2724,4501,3786,4350,7118,824,5981,1828,3554,6112,3880,3702,1174,4297,265,6994,1576,5493
3,3786,2994,554,1120,1811,5433,4074,3746,3847,1969,3238,2760,5210,5232,6852,6480,130,4767,1134,6127,5781,1232,5092,2289,3484,2099,1896,6478,5041,2207
4,4609,3120,4115,126,3843,2703,2145,257,6183,6918,5603,5246,2940,6200,2441,1283,5150,1509,1603,1622,6154,6165,1214,1368,3566,3271,6177,1043,1995,3886
5,1607,1438,6521,4876,5271,4278,117,4801,6221,3476,6323,4736,6278,5656,839,4322,6581,4698,4341,3134,1891,3214,1064,5012,3753,2499,2493,4727,1699,4580


In [17]:
# top 30 KNN for each movie
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)
sim_user_30_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,top11,top12,top13,top14,top15,top16,top17,top18,top19,top20,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1241,6019,1535,6505,1536,5956,4863,3354,7071,4179,2287,1013,1337,1410,4573,6856,3082,3044,522,6588,1517,3918,2251,800,3292,5053,3555,287,3952,4404
2,5975,4863,1535,7071,1536,3669,6505,3354,2287,1410,6019,5158,4099,1517,1576,3229,5519,6588,4422,5053,5956,4179,6591,800,5688,3305,2526,1337,3082,2704
3,5781,2207,1535,4863,1536,1740,6591,639,2861,6505,5426,2287,2884,1410,3952,6588,3082,7071,5158,3123,326,6019,3229,3489,6504,6192,7035,4259,5721,4573
4,126,4609,5987,6992,6671,1535,3354,2827,6154,4123,5317,1536,5617,1043,6505,2287,3319,4863,7035,5222,3120,6019,6591,1517,1410,7071,5603,6504,5158,6918
5,1438,6323,6278,4123,4863,4876,6290,5804,1535,5956,1013,6413,5222,6487,1536,3229,2422,6505,3134,3354,6521,2287,6019,2450,1517,7071,1410,6504,3380,6975


In [0]:
def get_user_similar_movies( user1, user2 ):
    common_movies = Rating_avg[Rating_avg.userId == user1].merge(
    Rating_avg[Rating_avg.userId == user2],
    on = "movieId",
    how = "inner" )
    return common_movies.merge( movies, on = 'movieId' )

In [20]:
a = get_user_similar_movies(360,1029)
a = a.loc[ : , ['rating_x_x','rating_x_y','title']]
a.head()

,rating_x_x,rating_x_y,title
0,5.0,4.0,Apollo 13 (1995)
1,5.0,5.0,"Shawshank Redemption, The (1994)"
2,4.0,5.0,"Fugitive, The (1993)"


In [0]:
def User_item_score(user,item):
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = final_movie.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_movie.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

In [22]:
#menampilkan score berdasarkan rating + jumlah adg score * correlation  / Jumlah correlation 
score = User_item_score(320,7371)
print("score (u,i) is",score)

score (u,i) is 4.197937016136487


In [0]:
Rating_avg = Rating_avg.astype({"movieId": str})
Movie_user = Rating_avg.groupby(by = 'userId')['movieId'].apply(lambda x:','.join(x))

In [0]:
#scrip memberikan rekomnedasi movie berdasarkan user ID
def User_item_score1(user):
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_movie.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_movie.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'movieId':Movies_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    Movie_Name = top_5_recommendation.merge(movies, how='inner', on='movieId')
    Movie_Names = Movie_Name.title.values.tolist()
    return Movie_Names

In [26]:
#menampilkan rekomendasi berdasarkan user ID
user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for User Id : 370")
print("   ")
for i in predicted_movies:
    print(i)

Enter the user id to whom you want to recommend : 75
 
The Recommendations for User Id : 370
   
Godfather, The (1972)
Usual Suspects, The (1995)
Schindler's List (1993)
Godfather: Part II, The (1974)
City of God (Cidade de Deus) (2002)
